In [3]:
import pandas as pd
import numpy as np
import matplotlib
import os

In [221]:
df_train = pd.read_csv("../data/train.csv", "|")
df_train_sorted = df_train.sort_values(by=['well id', 'depth, m'])
df_train_sorted.to_csv('../data/train_sorted.csv',index=False, sep='|')
df_train_sorted.to_json('../data/train_sorted.json')

In [20]:
df_train_clean = df_train.drop_duplicates(subset=['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS'], keep=False)
df_train_clean.to_csv('../data/train_clean.csv',index=False, sep='|')
df_train_clean.to_json('../data/train_clean.json')
df_train_clean_sorted = df_train_clean.sort_values(by=['well id', 'depth, m'])
df_train_clean_sorted.to_csv('../data/train_clean_sorted.csv',index=False, sep='|')
df_train_clean_sorted.to_json('../data/train_clean_sorted.json')

In [21]:
df_test = pd.read_csv("/Users/zamaletdinovaaa/mipt_cet_hack/Silicondzor/data/test.csv", "|")
df_test.to_json('../data/test.json')
df_test_sorted = df_test.sort_values(by=['well id', 'depth, m'])
df_test_sorted.to_json('../data/test_sorted.json')

In [237]:
_PNB = 4150
_PHI = 0.7
_RHO = 860
_S = 100
_H_DELTA = 0.1
_EXPENCES_BK = 245
_EXPENCES_GZ = 205
_EXPENCES_DGK = 130
_EXPENCES_NKT = 205
_EXPENCES_ALPS = 115
_EXPENCES_DICT = {'bk':_EXPENCES_BK, 
                 'GZ1':_EXPENCES_GZ,
                  'GZ2':_EXPENCES_GZ,
                  'GZ3':_EXPENCES_GZ,
                  'GZ4':_EXPENCES_GZ,
                  'GZ5':_EXPENCES_GZ,
                  'GZ6':_EXPENCES_GZ,
                  'GZ7':_EXPENCES_GZ,
                  'DGK':_EXPENCES_DGK,
                  'NKTD':_EXPENCES_NKT,
                  'NKTM':_EXPENCES_NKT,
                  'NKTR':_EXPENCES_NKT,
                  'ALPS':_EXPENCES_ALPS
                 }
_EXPENCES_COLUMNS_ALL = list(_EXPENCES_DICT.keys())

In [241]:
def analysis_columns(df):
    return list(pd.Index(_EXPENCES_COLUMNS_ALL).intersection(df.columns))
    
def add_expences(df):
    df_temp = df[analysis_columns(df)]
    df_temp = df_temp.notnull().astype('int')
    
    df['expences'] = (df_temp * [_EXPENCES_DICT.get(key) for key in analysis_columns(df)]).sum(axis=1)
    return df

In [312]:
def form_result_df(df):
    df_grouped = df.groupby(['well id'])
    columns = ['well id', 'total_cnt', 'oil_cnt', 'value', 'expenses', 'square', 'mass', 'price']

    df_dashbord_show=None
    df_dashbord_show = pd.DataFrame(columns=columns)
    
    for well_id, df_well in df_grouped:
        total_cnt = df_well.shape[0]
        oil_cnt = df_well[df_well['goal']==1]['well id'].count()
        h = _H_DELTA * oil_cnt
        expences = df_well['expences'].sum()
        square = oil_cnt*_S
        mass = h* square * _RHO
        value = _PNB*_PHI*h*_S*_RHO - expences
        price = mass * _PNB
        row_dict = [{'price':price,'mass':mass,'square':square,'well id':well_id, 'total_cnt':total_cnt, 'oil_cnt':oil_cnt, 'value':value, 'expenses':expences}]
        df_dashbord_show_inc = pd.DataFrame(row_dict)
        df_dashbord_show = df_dashbord_show.append(df_dashbord_show_inc)
    return df_dashbord_show

In [284]:
df_train_expences = add_expences(df_train)   
df_result_example = df_train_expences.iloc[:50]
df_result_example.to_json('../data/result_example_better.json',orient='split')

In [307]:
df_result_total_example=df_result_total_example.reset_index(drop=True)
df_result_total_example = form_result_df(df_result_example)

In [313]:
df_test_pred_full = pd.merge(df_test, df_test_pred, on='id')
df_test_pred_full_expences = add_expences(df_test_pred_full)
df_test_pred_full_expences.to_csv('../data/df_test_pred_full_expences.csv',index=False, sep='|')
df_test_pred_full_total = form_result_df(df_test_pred_full)
df_test_pred_full_total.to_csv('../data/df_test_pred_full_total.csv',index=False, sep='|')

In [11]:
df_grouped = df_test.groupby(['well id'])

for well_id, df_well in df_grouped:
    print(f'well_id {well_id} nunique - {df_well["depth, m"].nunique()} shape {df_well.shape}')
          
          
df_test[df_test.duplicated(['well id','depth, m','bk','GZ1','GZ2','GZ3','GZ4','GZ5','GZ7','DGK','NKTD','NKTM','NKTR','ALPS',], keep=False)]

well_id 47 nunique - 1297 shape (1360, 15)
well_id 95 nunique - 1162 shape (1199, 15)
well_id 126 nunique - 17382 shape (17633, 15)
well_id 164 nunique - 15826 shape (16238, 15)
well_id 176 nunique - 1136 shape (1174, 15)


,id,well id,"depth, m",bk,GZ1,GZ2,GZ3,GZ4,GZ5,GZ7,DGK,NKTD,NKTM,NKTR,ALPS
30,31,47,2468.231,0.449133,0.376967,0.127989,0.531776,0.253316,0.911877,0.417324,0.769427,0.381311,0.558993,0.422383,0.774000
31,32,47,2468.231,0.449133,0.376967,0.127989,0.531776,0.253316,0.911877,0.417324,0.769427,0.381311,0.558993,0.422383,0.774000
39,40,47,2468.947,0.458829,0.316576,0.117564,0.435454,0.354642,0.893757,0.369689,0.515214,0.381458,0.543162,0.411672,0.774000
40,41,47,2468.947,0.458829,0.316576,0.117564,0.435454,0.354642,0.893757,0.369689,0.515214,0.381458,0.543162,0.411672,0.774000
60,61,47,2470.737,0.367247,0.276679,0.119591,0.552777,0.114990,0.223684,0.645786,0.366495,0.386175,0.563918,0.425497,0.368554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37520,37521,176,2165.005,0.399551,0.262832,0.127844,0.782174,0.141857,0.368494,0.852176,0.494653,0.347186,0.498485,0.424384,0.162172
37528,37529,176,2165.721,0.423699,0.247487,0.131655,0.675283,0.125893,0.371813,0.841864,0.412178,0.363219,0.513265,0.496400,0.132560
37529,37530,176,2165.721,0.423699,0.247487,0.131655,0.675283,0.125893,0.371813,0.841864,0.412178,0.363219,0.513265,0.496400,0.132560
37565,37566,176,2168.943,0.397237,0.279274,0.154195,0.733595,0.117446,0.357766,0.937225,0.344939,0.393485,0.527332,0.379199,0.195392


In [314]:
df_test_pred = pd.read_csv("../data/predicted_test.csv", ",")

In [302]:
(df_test_pred.shape, df_test.shape)

((37604, 3), (37604, 15))

In [322]:
print(f"mass {df_test_pred_full_total['mass'].sum()}")

mass 154291619600.0


In [324]:
df_test_pred_full_total
pd.set_option('display.float_format', '{:.2f}'.format)

In [325]:
df_test_pred_full_total

,expenses,mass,oil_cnt,price,square,total_cnt,value,well id
0,3175600,897229400.00,323,3723502010000.00,32300,1360,8066333400.00,47
0,2799550,248540000.00,170,1031441000000.00,17000,1199,4244310450.00,95
0,41169250,65369021400.00,2757,271281438810000.00,275700,17633,68836961750.00,126
0,36182310,87404758400.00,3188,362729747360000.00,318800,16238,79609621690.00,164
0,2741290,372070400.00,208,1544092160000.00,20800,1174,5193722710.00,176


In [262]:
df_test_pred_full_total['oil_cnt'].sum(), df_test_pred['goal'].sum()

(6646, 6646)

In [258]:
g1 = df_test_pred_full_expences[['well id', 'depth, m', 'goal', 'lith']] 

g1.to_csv('../data/df_test_pred_full_expences_1g.csv',index=False, sep='|')


In [311]:
df_test_pred_full_total['price'] - df_test_pred_full_total['expenses']

0    1.15247e+10
0     6.0645e+09
0    9.83562e+10
0    1.13744e+11
0    7.42078e+09
dtype: object

In [298]:
df_test_pred_full_total['value'] - df_test_pred_full_total['price']

0    2.525725e+11
0    6.795096e+10
0    1.892078e+13
0    2.531140e+13
0    1.028872e+11
dtype: float64